In [39]:
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", category=FutureWarning)
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import impute
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV, LogisticRegressionCV, LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, \
cross_validate, GridSearchCV, RandomizedSearchCV, KFold, StratifiedKFold, RepeatedKFold, RepeatedStratifiedKFold
from skopt import BayesSearchCV
import ast
import itertools
import os

os.getcwd()
os.chdir('/Users/kevin/Downloads/Northwestern University/Data Science/STAT_303-3/Prediction Problems')

## Step 0) Reading the data

In [2]:
train = pd.read_csv('train_classification.csv')
test = pd.read_csv('test_classification.csv')

## Step 1) Data pre-processing

### <font color = 'red'>Pre-processing training data</font>

In [3]:
first_ten = train.iloc[:, :10]

# Removing: ['id', 'host_location', 'host_neighbourhood']
cleaned_ten = first_ten.drop(columns=['id', 'host_location', 'host_neighbourhood'])

# Converting: ['host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_since']
cleaned_ten['host_response_rate'] = pd.to_numeric(cleaned_ten['host_response_rate'].str.strip('%')) / 100
cleaned_ten['host_acceptance_rate'] = pd.to_numeric(cleaned_ten['host_acceptance_rate'].str.strip('%')) / 100
cleaned_ten['host_is_superhost'] = cleaned_ten['host_is_superhost'].map({'t': 1, 'f': 0})
cleaned_ten['host_since'] = pd.to_datetime(cleaned_ten['host_since'])
cleaned_ten['days_since_host'] = (pd.datetime.now() - cleaned_ten['host_since']).dt.days
cleaned_ten = cleaned_ten.drop(columns=['host_since'])



second_ten = train.iloc[:, 10:20]

# Converting: ['host_has_profile_pic', 'neighbourhood_cleansed', 'host_identity_verified','latitude', 'longitude', 'property_type', 'room_type']
cleaned_twenty = second_ten
neighbourhood_counts = cleaned_twenty.neighbourhood_cleansed.value_counts()
neighbourhoods_to_replace = neighbourhood_counts[neighbourhood_counts < 107].index.tolist()
cleaned_twenty['neighbourhood_cleansed'] = cleaned_twenty['neighbourhood_cleansed'].replace(neighbourhoods_to_replace, 'Other')
cleaned_twenty['num_verifications'] = cleaned_twenty['host_verifications'].apply(lambda x: len(ast.literal_eval(x)))
cleaned_twenty = cleaned_twenty.drop(columns=['host_verifications'])
cleaned_twenty['host_has_profile_pic'] = cleaned_twenty['host_has_profile_pic'].map({'t': 1, 'f': 0})
cleaned_twenty['host_identity_verified'] = cleaned_twenty['host_identity_verified'].map({'t': 1, 'f': 0})
cleaned_twenty['latitude'] = pd.to_numeric(cleaned_twenty['latitude'])
cleaned_twenty['longitude'] = pd.to_numeric(cleaned_twenty['longitude'])
cleaned_twenty['property_category'] = "Entire property"
cleaned_twenty.loc[cleaned_twenty['property_type'].str.contains('room', case=False), 'property_category'] = 'Room'
cleaned_twenty = cleaned_twenty.drop(columns=['property_type'])



third_ten = train.iloc[:, 20:30]

# Converting: ['bathrooms_text', 'price']
third_ten['bathrooms_text'] = third_ten['bathrooms_text'].replace({"Half-bath": "0.5", "Shared half-bath": "0.5", "Private half-bath": "0.5"})
third_ten['num_bathrooms'] = third_ten['bathrooms_text'].str.extract(r'(\d+(\.\d+)?)')[0].astype(float)
cleaned_third = third_ten.drop(columns=['bathrooms_text'])



fourth_ten = train.iloc[:, 30:40]

# Removing: ['first_review', 'last_review']
cleaned_fourth = fourth_ten.drop(columns=['first_review'])
cleaned_fourth = cleaned_fourth.drop(columns=['last_review'])

# Converting: ['has_availability']
cleaned_fourth['has_availability'] = cleaned_fourth['has_availability'].map({'t': 1, 'f': 0})



fifth_ten = train.iloc[:, 40:50]
fifth_ten

# Converting: ['instant_bookable']
cleaned_fifth = fifth_ten
cleaned_fifth['instant_bookable'] = cleaned_fifth['instant_bookable'].map({'t': 1, 'f': 0})


last_three = train.iloc[:, 50:]

In [4]:
# Combining the cleaned datasets
cleaned_train = pd.concat([cleaned_ten, cleaned_twenty, cleaned_third, cleaned_fourth, cleaned_fifth, last_three], axis=1)

In [5]:
# Imputing missing values

columns_with_missing = ['host_response_rate', 'host_acceptance_rate', 'beds', \
                        'num_bathrooms', 'review_scores_rating']

cleaned_train['reviews_per_month'].fillna(cleaned_train['reviews_per_month'].mode()[0], inplace=True)
cleaned_train['host_response_time'].fillna(cleaned_train['host_response_time'].mode()[0], inplace=True)

# Computing the missing values of numeric variables using KNN

knn_imputer = impute.KNNImputer(n_neighbors=10)
cleaned_train_imputed = knn_imputer.fit_transform(cleaned_train[columns_with_missing])
cleaned_train_imputed_df = pd.DataFrame(cleaned_train_imputed, columns=columns_with_missing)
cleaned_train[columns_with_missing] = cleaned_train_imputed_df

In [6]:
to_be_removed = ['review_scores_communication','review_scores_cleanliness', 'number_of_reviews_l30d', \
                                'review_scores_accuracy', 'review_scores_value','review_scores_location', \
                                'review_scores_checkin', 'minimum_minimum_nights', 'maximum_minimum_nights', \
                                'minimum_maximum_nights', 'maximum_maximum_nights', 'availability_60', \
                                'availability_90', 'availability_365','calculated_host_listings_count',
                                'calculated_host_listings_count_entire_homes', 'host_listings_count']

In [7]:
y_train = cleaned_train.host_is_superhost
X_train = cleaned_train.drop("host_is_superhost", axis = 1).iloc[:, 1:]

In [8]:
X_train_non_redundant = X_train.copy()
X_train_non_redundant.drop(columns = to_be_removed, inplace = True)
X_train_non_redundant = pd.get_dummies(X_train_non_redundant, drop_first = True)

#### PolynomialFeatures

In [9]:
poly = PolynomialFeatures(2, interaction_only = True, include_bias = False)
X_train_poly = poly.fit_transform(X_train_non_redundant)

In [10]:
X_train_non_scaled_poly_df = pd.DataFrame(X_train_poly, columns = poly.get_feature_names_out(X_train_non_redundant.columns))

#### Scaling

In [11]:
scaler = StandardScaler().fit(X_train_poly)
X_train_scaled = scaler.transform(X_train_poly)

### <font color = 'red'>Pre-processing test data</font>

In [50]:
first_ten = test.iloc[:, :10]

# Removing: ['id', 'host_id', 'host_since', 'host_location', 'host_response_time', 'host_neighbourhood']
cleaned_ten = first_ten.drop(columns=['host_id', 'host_location', 'host_neighbourhood'])

# Converting: ['host_response_rate', 'host_acceptance_rate', 'host_is_superhost']
cleaned_ten['host_response_rate'] = pd.to_numeric(cleaned_ten['host_response_rate'].str.strip('%')) / 100
cleaned_ten['host_acceptance_rate'] = pd.to_numeric(cleaned_ten['host_acceptance_rate'].str.strip('%')) / 100
cleaned_ten['host_since'] = pd.to_datetime(cleaned_ten['host_since'])
cleaned_ten['days_since_host'] = (pd.datetime.now() - cleaned_ten['host_since']).dt.days
cleaned_ten = cleaned_ten.drop(columns=['host_since'])



second_ten = test.iloc[:, 10:20]

# Converting: ['host_has_profile_pic', 'neighbourhood_cleansed', 'host_verifications', 'host_identity_verified','latitude', 'longitude', 'property_type', 'room_type']
cleaned_twenty = second_ten
neighbourhood_counts = cleaned_twenty.neighbourhood_cleansed.value_counts()
neighbourhoods_to_replace = neighbourhood_counts[neighbourhood_counts < 64].index.tolist()
cleaned_twenty['neighbourhood_cleansed'] = cleaned_twenty['neighbourhood_cleansed'].replace(neighbourhoods_to_replace, 'Other')
cleaned_twenty['num_verifications'] = cleaned_twenty['host_verifications'].apply(lambda x: len(ast.literal_eval(x)))
cleaned_twenty = second_ten.drop(columns=['host_verifications'])
cleaned_twenty['host_has_profile_pic'] = cleaned_twenty['host_has_profile_pic'].map({'t': 1, 'f': 0})
cleaned_twenty['host_identity_verified'] = cleaned_twenty['host_identity_verified'].map({'t': 1, 'f': 0})
cleaned_twenty['latitude'] = pd.to_numeric(cleaned_twenty['latitude'])
cleaned_twenty['longitude'] = pd.to_numeric(cleaned_twenty['longitude'])
cleaned_twenty['bathrooms_text'] = cleaned_twenty['bathrooms_text'].replace({"Half-bath": "0.5", "Shared half-bath": "0.5", "Private half-bath": "0.5"})
cleaned_twenty['num_bathrooms'] = cleaned_twenty['bathrooms_text'].str.extract(r'(\d+(\.\d+)?)')[0].astype(float)
cleaned_twenty = cleaned_twenty.drop(columns=['bathrooms_text'])

cleaned_twenty['property_category'] = "Entire property"
cleaned_twenty.loc[cleaned_twenty['property_type'].str.contains('room', case=False), 'property_category'] = 'Room'
cleaned_twenty = cleaned_twenty.drop(columns=['property_type'])




third_ten = test.iloc[:, 20:30]

# Converting: ['has_availability']
third_ten['has_availability'] = third_ten['has_availability'].map({'t': 1, 'f': 0})




fourth_ten = test.iloc[:, 30:40]

# Removing: ['first_review', 'last_review']
# Converting: []
cleaned_fourth = fourth_ten.drop(columns=['first_review', 'last_review'])



fifth_ten = test.iloc[:, 40:50]
# Converting: ['instant_bookable']
cleaned_fifth = fifth_ten
cleaned_fifth['instant_bookable'] = cleaned_fifth['instant_bookable'].map({'t': 1, 'f': 0})



last_three = test.iloc[:, 50:]

In [51]:
# Combining test data
cleaned_test = pd.concat([cleaned_ten, cleaned_twenty, third_ten, cleaned_fourth, cleaned_fifth, last_three], axis=1)

In [52]:
# Imputing missing values

columns_with_missing = ['host_response_rate', 'host_acceptance_rate', 'beds', \
                        'num_bathrooms', 'review_scores_rating']

cleaned_test['reviews_per_month'].fillna(cleaned_test['reviews_per_month'].mode()[0], inplace=True)
cleaned_test['host_response_time'].fillna(cleaned_test['host_response_time'].mode()[0], inplace=True)

# Computing the missing values of numeric variables using KNN

knn_imputer = impute.KNNImputer(n_neighbors=10)
cleaned_test_imputed = knn_imputer.fit_transform(cleaned_test[columns_with_missing])
cleaned_test_imputed_df = pd.DataFrame(cleaned_test_imputed, columns=columns_with_missing)
cleaned_test[columns_with_missing] = cleaned_test_imputed_df

In [53]:
X_test_non_redundant = cleaned_test.drop(columns = to_be_removed)
X_test_non_redundant = pd.get_dummies(X_test_non_redundant, drop_first = True)
X_test_non_redundant = X_test_non_redundant.iloc[:, 1:]

#### <font color = 'red'>Polynomial Features</font>

In [54]:
poly_test = PolynomialFeatures(2, include_bias = False)
poly_test.fit(X_test_non_redundant)
X_test_poly = poly_test.transform(X_test_non_redundant)

In [55]:
X_test_non_scaled_poly_df = pd.DataFrame(X_test_poly, columns = poly_test.get_feature_names_out(X_test_non_redundant.columns))

## Step 2) Hyperparameter tuning

### How many attempts did it take you to tune the model hyperparameters?

It took me two attempts to tune the model hyperparameters. My first attempt was performed without variable interactions, which resulted in an accuracy of around 88%. Since this was already pretty close to the threshold accuracy, I predicted that using `PolynomialFeatures` would be able to improve my accuracy to over 90%. Although utilizing variable interactions greatly increased the amount of time it took for Lasso to eliminate predictors, it did end up helping me reach the threshold accuracy on my second attempt.

### Which tuning method did you use?

I used random search (`RandomizedSearchCV`) to tune the hyperparameters.

### What challenges did you face while tuning the hyperparameters, and what actions did you take to address those challenges?

The main challenge that I faced while tuning the hyperparameters was creating the grid over which the search method would be performed. Since the tuning process took so long to complete, I minimized the number of values for each hyperparameter that I aimed to tune. As a result, I decided to use the `RandomizedSearchCV` search method to tune hyperparameters, as it would be more efficient than using `GridSearchCV`, while only testing for 20 possible number of neighbors and two possible weights. Even though my search grid was not very large, it was still able to achieve an accuracy of over 90% on Kaggle.

### How many hours did you spend on hyperparameter tuning?

It took about 2 hours for the code to finish tuning the hyperparameters. I spent about 90 minutes on the code that performs the hyperparameter tuning.

### Hyperparameter tuning code

#### <font color = 'red'>Lasso for variable selection</font>

In [18]:
Cs = np.logspace(-1,-5, 200)
model_cv = LogisticRegressionCV(Cs = Cs, cv=10, penalty='l1', solver = 'saga')
model_cv.fit(X_train_scaled, y_train)

LogisticRegressionCV(Cs=array([1.00000000e-01, 9.54771611e-02, 9.11588830e-02, 8.70359136e-02,
       8.30994195e-02, 7.93409667e-02, 7.57525026e-02, 7.23263390e-02,
       6.90551352e-02, 6.59318827e-02, 6.29498899e-02, 6.01027678e-02,
       5.73844165e-02, 5.47890118e-02, 5.23109931e-02, 4.99450512e-02,
       4.76861170e-02, 4.55293507e-02, 4.34701316e-02, 4.15040476e-02,
       3.96268864e-02, 3.78...
       2.89942285e-05, 2.76828663e-05, 2.64308149e-05, 2.52353917e-05,
       2.40940356e-05, 2.30043012e-05, 2.19638537e-05, 2.09704640e-05,
       2.00220037e-05, 1.91164408e-05, 1.82518349e-05, 1.74263339e-05,
       1.66381689e-05, 1.58856513e-05, 1.51671689e-05, 1.44811823e-05,
       1.38262217e-05, 1.32008840e-05, 1.26038293e-05, 1.20337784e-05,
       1.14895100e-05, 1.09698580e-05, 1.04737090e-05, 1.00000000e-05]),
                     cv=10, penalty='l1', solver='saga')

In [19]:
coefficients = {}
for i in range(len(model_cv.coef_[0])):
    coefficients[poly.get_feature_names_out()[i]] = model_cv.coef_[0][i]

In [20]:
sorted_predictors = pd.Series(coefficients).sort_values(key = abs, ascending = False).index.to_list()

#### <font color = 'red'>Tuning the Hyperparameters</font>

In [65]:
analysis_results = pd.DataFrame(columns = ['r', 'predictors', 'n_neighbors', 'weights', 'p', 'threshold', 'accuracy'])

In [66]:
warnings.simplefilter(action='ignore', category=FutureWarning)

threshold_vals = np.arange(0, 1.01, 0.01)

for i in range(20, 51):
    predictors = sorted_predictors[:i]
    X = X_train_non_scaled_poly_df.loc[:, predictors]
    sc = StandardScaler()
    Xstd = sc.fit_transform(X)

    # Using RandomizedSearchCV to tune the hyperparameters:
    
    # 1) Create the model
    model = KNeighborsClassifier(metric='minkowski')

    # 2) Create a hyperparameter grid (as a dict)   
    grid = {'n_neighbors': np.arange(1, 21), 'weights': ['uniform', 'distance'], 'p': [1]}

    # 3) Create the Kfold object
    kfold = KFold(n_splits=5, shuffle=True, random_state=1)

    # 4) Create the CV object
    gcv = RandomizedSearchCV(model, param_distributions=grid, cv=kfold, n_iter=500, scoring='accuracy', n_jobs=-1)

    # Fit the models, and cross-validate
    gcv.fit(Xstd, y_train)
    
    # Tune threshold parameters within the same loop
    predicted_probability = cross_val_predict(KNeighborsClassifier(n_neighbors=gcv.best_params_['n_neighbors'], weights=gcv.best_params_['weights'], p=gcv.best_params_['p'], metric='minkowski'), Xstd, y_train, cv=kfold, method='predict_proba')
    for threshold_prob in threshold_vals:
        predicted_class = predicted_probability[:, 1] > threshold_prob
        predicted_class = predicted_class.astype(int)
        accuracy = accuracy_score(predicted_class, y_train) * 100
        analysis_results = analysis_results.append({'r': i, 'predictors': predictors, 'n_neighbors': gcv.best_params_['n_neighbors'], 'weights': gcv.best_params_['weights'], 'p': gcv.best_params_['p'], 'threshold': threshold_prob, 'accuracy': accuracy}, ignore_index=True)

/Users/kevin/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/kevin/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer b

/Users/kevin/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/kevin/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer b

/Users/kevin/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/kevin/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer b

/Users/kevin/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/kevin/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer b

/Users/kevin/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/kevin/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer b

...

/Users/kevin/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/kevin/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer b

/Users/kevin/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/kevin/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer b

/Users/kevin/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/kevin/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer b

/Users/kevin/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/kevin/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer b

/Users/kevin/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/kevin/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer b

### Optimal hyperparameter values

In [67]:
analysis_results.sort_values(by='accuracy', ascending=False).iloc[0]

r                                                             48
predictors     [number_of_reviews_ltm review_scores_rating, h...
n_neighbors                                                   11
weights                                                 distance
p                                                              1
threshold                                                   0.55
accuracy                                               81.354229
Name: 2883, dtype: object

The optimal value of `n_neighbors` is 11, the optimal number of predictors is 48, the optimal `weights` is `distance`, the optimal value of `p`is 1, and the optimal decision threshold is 0.55.

## Step 3) Developing the model

In [56]:
# sorted_predictors is defined in Step 2)
predictors = sorted_predictors[:48]
Xtrain = X_train_non_scaled_poly_df.loc[:, predictors]
Xtest = X_test_non_scaled_poly_df.loc[:, predictors]
sc  = StandardScaler()
Xtrain_std = sc.fit_transform(Xtrain)
Xtest_std = sc.transform(Xtest)

In [57]:
model = KNeighborsClassifier(n_neighbors = 11, metric = 'minkowski', weights = 'distance', p = 1).fit(Xtrain_std, y_train)

# Making the prediction
y_pred = model.predict_proba(Xtest_std)
predicted_class = y_pred[:,1] > 0.55
test_pred = predicted_class.astype(int)

In [58]:
cleaned_test.insert(1, "predicted", test_pred)
to_submit = cleaned_test.iloc[:, :2]

## Step 4) Ad-hoc steps for further improving model accuracy

#### <font color = 'red'>Matching known host_id's</font>

In [59]:
known = train.loc[:, ['id', 'host_id','host_is_superhost']]
known['host_is_superhost'] = known['host_is_superhost'].map({'t': 1, 'f': 0})

not_known = test.loc[:, ['id', 'host_id']]

overlapping_host_ids = set(known['host_id']).intersection(set(not_known['host_id']))

In [60]:
for host_id in overlapping_host_ids:
    host_is_superhost = known.loc[known['host_id'] == host_id, 'host_is_superhost'].iloc[0]
    not_known.loc[not_known['host_id'] == host_id, 'host_is_superhost'] = host_is_superhost

In [61]:
merged_df = pd.merge(to_submit, not_known[['id', 'host_is_superhost']], on='id', how='left')
merged_df.host_is_superhost = merged_df.host_is_superhost.fillna(merged_df.predicted)

## Step 5) Exporting the predictions in the format required to submit on Kaggle

In [62]:
final_submission = merged_df.iloc[:, [0,2]].rename(columns = {'host_is_superhost':'predicted'})

In [63]:
final_submission.to_csv('KNN Classification - Final Submission.csv', index=False)  